In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리하세요.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=1)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 538, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 538 (delta 165), reused 184 (delta 107), pack-reused 266
Receiving objects: 100% (538/538), 46.22 MiB | 13.98 MiB/s, done.
Resolving deltas: 100% (264/264), done.
/content/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.26.0
Using datasets v2.9.0
Using accelerate v0.16.0
Using sentencepiece v0.1.97
Using sacremoses v0.0.41
No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Go to Runtime > Change runtime type and select a GPU hardware accelerator.


In [ ]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

## 텍스트 분류

pipeline 코드를 실행하면 파이프라인이 자동으로 허깅페이스 허브에서 모델 가중치를 다운로드함 -> 몇 개의 진행 표시줄이 나타남.  
기본적으로 text-classification 파이프라인은 감성 분석을 위해 설계된 모델을 사용하지만, 다중 분류와 다중 레이블 분류 또한 지원함.

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification") #파이프라인 - 텍스트 분류

파이프라인이 준비됐으니 예측을 만들어보자!  
각 파이프라인은 텍스트 문자열(또는 문자열의 리스트)를 입력으로 받고 예측 리스트를 반환함.

In [ ]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


모델은 텍스트가 부정적이라고 확신함. 즉 화난 고객의 불만을 처리하고 있다는 뜻.

## 개체명 인식

고객 피드백의 감성을 예측하는 것은 중요하지만,, 피드백이 특정 제품과 서비스 중 무엇에 대한 것인지 알고 싶을 경우가 많음.  
NLP에서는 제품, 장소, 사람과 같은 실제 객체를 개체명이라고 하고, 이런 개체명을 텍스트에서 추출하는 작업을 개체명인식(NER)이라고 함.  

해당 파이프라인을 로드하고 고객 리뷰를 전달해 NER을 적용해보자!

In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy = "simple") #파이프라인 - ner(개체명 인식) / aggregation_strategy : 모델 예측에 따라 단어를 그룹화
outputs = ner_tagger(text)
pd.DataFrame(outputs)

,entity_group,score,word,start,end
0,ORG,0.879011,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


해당 파이프라인(NER)은 모든 개체명을 감지하고 ORG(조직), LOC(위치), PER(사람) 같은 카테고리에 할당함. (MISC : 그 외)    

점수는 모델이 개체명을 얼마나 확신하는지를 나타냄. 
'Decepticons'와 'Megatron'에서 확산이 가장 낮음. 따라서 두 경우 모두 하나의 개체로 묶이지 못함.

텍스트에 있는 모든 개체명이 잘 추출되었다. 하지만 더 구체적인 질문을 하고 싶을 때가 있다. 이를 위해 질문답변을 사용하자

# 질문 답변
질문 답변에서는 텍스트 구절과 함께 답을 얻고 싶은 질문을 모델에 전달하고, 모델은 답변을 텍스트로 반환함. 고객의 피드백에 대해 질문했을 때 어떤 답이 나오는지 확인하자.

In [ ]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question = question, context = text)
pd.DataFrame([outputs])

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


파이프라인은 답변과 함께 (NER처럼) 답이 위치한 문자 인덱스에 해당하는 start와 end 정수도 반환함.  
해당 예제의 경우 답변을 텍스트에서 직접 추출하기에 **추출적 질문답변**이라고 함.

## 텍스트 요약  
텍스트 요약의 목표는 긴 텍스트를 입력으로 받고 관련 사실이 모두 포함된 간단한 버전을 생성하는 것. 모델이 논리적인 텍스트를 생성해야 하므로 이전 문제보다 훨씬 복잡한 작업임.

In [ ]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length = 60, clean_up_tokenization_spaces = True)
print(outputs[0]['summary_text'])

 Bumblebee ordered an Optimus Prime action figure from your online store in
Germany. Unfortunately, when I opened the package, I discovered to my horror
that I had been sent an action figure of Megatron instead. As a lifelong enemy
of the Decepticons, I hope you can understand


요약 결과가 나쁘지 않게 나왔다. 원본이 일부 복사됐으나 모델이 문제의 핵심을 감지하고, 'Bumblebee'가 불만을 제기한 사람이라는 것을 명확하게 구분해냈다.  
해당 예시에서는 파이프라인에 max_length와 clean_up_tokenization_spaces 키워드 매개변수를 사용했다. 이러한 매개변수를 통해 실행 시점에 출력을 조정한다.  

만약 피드백이 이해할 수 없는 언어로 되어있다면?? 구글 번역을 이용하거나 직접 트랜스포머 모델을 사용해 번역하면 된다.

## 번역
요약과 마찬가지로, 번역 또한 텍스트를 생성해 출력한다. 번역 파이프라인을 사용해 영어 텍스트를 독일어로 번역해보자.

In [ ]:
translator = pipeline('translation_en_to_de', model = 'Helsinki-NLP/opus-mt-en-de')
outputs = translator(text, clean_up_tokenization_spaces = True, min_length = 100)
print(outputs[0]['translation_text'])

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus
Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete,
entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von
Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich
hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere
einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt.
Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von
Ihnen zu hören. Aufrichtig, Bumblebee.


매우 번역이 잘 되었다. 독일어 대명사 또한 바르게 사용되었다. 또한 파이프라인의 기본 모델을 오버라이드해서 애플리케이션에 가장 잘 맞는 모델을 선택하는 방법을 보여준다.

## 텍스트 생성
자동 완성 기능으로 고객 피드백에 빠르게 응답하고 싶다고 가정해보자. 이때 텍스트 생성 모델을 사용한다.

In [ ]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response;\n" + response #???
outputs = generator(prompt, max_length = 200)
print(outputs[0]['generated_text'])

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Dear Amazon, last week I ordered an Optimus Prime action figure from your online
store in Germany. Unfortunately, when I opened the package, I discovered to my
horror that I had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the
issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear
from you soon. Sincerely, Bumblebee.

Customer service responese;
Dear Bumblebee, I am sorry to hear that your order was mixed up. I will make the
right decision and return both sets to you in due time. The first Optimus Prime
action figure is sent as a direct shipment and will arrive later on in the UK. I
have heard you are working closely with your suppliers and are pleased to inform
you of a delay between shipments. As you must recall, some


해당 답변으로 범블비를 진정시키지는 못하겠지만,,, 어떻게 답변해야할지 아이디어를 얻었다.